# Asynchronous Programming in Scala

This notebook demonstrates various concepts of asynchronous programming in Scala, comparing them with Rust's approach where relevant.

## Key Concepts

- **Threads**: OS-level constructs for parallel execution
- **Futures**: Lightweight abstractions for asynchronous computations
- **Eager vs Lazy Execution**: How Scala differs from Rust
- **Error Handling**: Managing failures in asynchronous code
- **Thread Pools**: Configuring execution contexts

![Components](thread-components.png)

![Hierarchy](thread-task-hierarchy.png)

Hierarchy: see file async_hierarchy_panics.scala

Let's start by importing the necessary dependencies:

In [1]:
import sys.process._
import scala.concurrent._
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global
import java.time.{Duration => JDuration, Instant}
import java.util.concurrent.{Executors, CopyOnWriteArrayList}
import scala.util.{Try, Success, Failure}

import sys.process._

import scala.concurrent._

import scala.concurrent.duration._

import scala.concurrent.ExecutionContext.Implicits.global

import java.time.{Duration => JDuration, Instant}

import java.util.concurrent.{Executors, CopyOnWriteArrayList}

import scala.util.{Try, Success, Failure}


## 1. Eager vs Lazy Execution

One fundamental difference between Scala's `Future` and Rust's async/await is **execution behavior**:

- **Scala Futures are eager**: They start executing as soon as they're created
- **Rust futures are lazy**: They only start when explicitly awaited

**File: `await.scala`**

In [2]:
// EAGER FUTURES (DEFAULT IN SCALA)
def lionRuns(): Future[Unit] = Future {
  println("🦁 Lion is running!")
}

def foxRuns(): Future[Unit] = Future {
  println("🦊 Fox is running!")
}

def rabbitRuns(): Future[Unit] = Future {
  println("🐇 Rabbit is running!")
}

println("🦁 lionRuns() is called.")
val lionFuture = lionRuns() // Execution begins immediately!
println("🦊 foxRuns() is called.")
val foxFuture = foxRuns()   // Execution begins immediately!
println("🐇 rabbitRuns() is called.")
val rabbitFuture = rabbitRuns() // Execution begins immediately!
println("(All animals are running already in Scala Future!)")

// We only await some of the futures
Await.result(foxFuture, 2.seconds)
Await.result(lionFuture, 2.seconds)

// We never await the rabbit, but it still runs!
println("All animals have finished running (even the rabbit 🐇, because Scala Future is eager)!")

🦁 lionRuns() is called.
🦊 foxRuns() is called.
🦁 Lion is running!
🐇 rabbitRuns() is called.
🦊 Fox is running!
(All animals are running already in Scala Future!)
🐇 Rabbit is running!
All animals have finished running (even the rabbit 🐇, because Scala Future is eager)!


defined function lionRuns
defined function foxRuns
defined function rabbitRuns
lionFuture: Future[Unit] = Success(value = ())
foxFuture: Future[Unit] = Success(value = ())
rabbitFuture: Future[Unit] = Success(value = ())

### Scala Futures: Key Patterns and Best Practices

In [2]:
import scala.concurrent.Future
import scala.concurrent._
import ExecutionContext.Implicits.global
import scala.util.{Failure, Success}
import scala.concurrent.duration._

// 1. CREATING FUTURES
println("1. Creating a basic Future:")
val basicFuture: Future[Int] = Future {
  println("  → Computing a result...")
  Thread.sleep(1000) // Simulate work
  42 // The result
}

// 2. CALLBACKS - Non-blocking way to handle results
println("\n2. Using callbacks (non-blocking):")
basicFuture.onComplete {
  case Success(value) => println(s"  → Success callback: The result is: $value")
  case Failure(exception) => println(s"  → Failure callback: An error occurred: ${exception.getMessage}")
}

// 3. TRANSFORMATIONS - Chaining operations
println("\n3. Transforming futures:")
val transformedFuture = basicFuture
  .map(result => result * 2)
  .map(doubled => s"The doubled result is: $doubled")

transformedFuture.onComplete {
  case Success(message) => println(s"  → $message")
  case Failure(exception) => println(s"  → Transform failed: ${exception.getMessage}")
}

// 4. COMPOSING FUTURES
println("\n4. Composing multiple futures:")
def getUsername(): Future[String] = Future { "user123" }
def getUserData(username: String): Future[Map[String, String]] = Future { 
  Map("name" -> "John Doe", "username" -> username) 
}

// Using flatMap for sequential composition
val userDataFuture = getUsername().flatMap(username => getUserData(username))

// The same thing using for-comprehension (more readable)
val userDataFuture2 = for {
  username <- getUsername()
  userData <- getUserData(username)
} yield userData

userDataFuture.onComplete {
  case Success(data) => println(s"  → User data: $data")
  case Failure(e) => println(s"  → Failed to get user data: ${e.getMessage}")
}

// 5. ERROR HANDLING
println("\n5. Error handling:")
val failingFuture: Future[String] = Future {
  throw new RuntimeException("Something went wrong!")
  "This will never be returned"
}

failingFuture
  .recover { case ex: RuntimeException => s"Recovered from: ${ex.getMessage}" }
  .onComplete {
    case Success(result) => println(s"  → Recovery result: $result")
    case Failure(ex) => println(s"  → Recovery failed: ${ex.getMessage}")
  }

// 6. AWAITING RESULTS (FOR DEMONSTRATION ONLY)
println("\n6. Awaiting results (avoid in production code):")
try {
  val result = Await.result(basicFuture, 2.seconds)
  println(s"  → Await result: $result")
} catch {
  case e: TimeoutException => println("  → Future timed out!")
  case e: Exception => println(s"  → Future failed: ${e.getMessage}")
}

println("\n=== BEST PRACTICES ===")
println("• Avoid blocking with Await.result in production code")
println("• Use callbacks and transformations (map/flatMap) instead")
println("• Handle errors explicitly with recover or recoverWith")
println("• Use for-comprehensions for multiple sequential futures")
println("• Consider custom ExecutionContexts for production apps")

// Sleep to ensure callbacks have time to execute
Thread.sleep(2000)

1. Creating a basic Future:

2. Using callbacks (non-blocking):
  → Computing a result...

3. Transforming futures:

4. Composing multiple futures:

5. Error handling:
  → User data: Map(name -> John Doe, username -> user123)

6. Awaiting results (avoid in production code):
  → Recovery result: Recovered from: Something went wrong!
  → Success callback: The result is: 42
  → Await result: 42
  → The doubled result is: 84

=== BEST PRACTICES ===
• Avoid blocking with Await.result in production code
• Use callbacks and transformations (map/flatMap) instead
• Handle errors explicitly with recover or recoverWith
• Use for-comprehensions for multiple sequential futures
• Consider custom ExecutionContexts for production apps


import scala.concurrent.Future

import scala.concurrent._

import ExecutionContext.Implicits.global

import scala.util.{Failure, Success}

import scala.concurrent.duration._

// 1. CREATING FUTURES

basicFuture: Future[Int] = Success(value = 42)
transformedFuture: Future[String] = Success(value = "The doubled result is: 84")
defined function getUsername
defined function getUserData
userDataFuture: Future[Map[String, String]] = Success(value = Map("name" -> "John Doe", "username" -> "user123"))
userDataFuture2: Future[Map[String, String]] = Success(value = Map("name" -> "John Doe", "username" -> "user123"))
failingFuture: Future[String] = Failure(exception = java.lang.RuntimeException: Something went wrong!)

## 2. Thread Pool Effects on Performance

Thread pool size affects how many CPU-bound tasks can execute in parallel. This example demonstrates the effect of different thread pool sizes.

**File: `async_threadpool.scala`**

In [4]:
// Define our CPU-bound task simulator
def cpuBoundTask(name: String): Unit = {
  println(s"$name starting work...")
  Thread.sleep(1000) // Simulate CPU-bound work
  println(s"$name finished!")
}

// Demo with 1 thread
def runWithThreads(threadCount: Int): Unit = {
  println(s"\n=== Running with $threadCount thread(s) ===\n")
  val start = Instant.now()
  
  // Create custom thread pool
  val executorService = Executors.newFixedThreadPool(threadCount)
  val ec = ExecutionContext.fromExecutorService(executorService)
  
  try {
    // Create three futures for our animals
    implicit val executionContext = ec
    
    val lion = Future {
      cpuBoundTask("🦁 Lion")
    }
    
    val fox = Future {
      cpuBoundTask("🦊 Fox")
    }
    
    val rabbit = Future {
      cpuBoundTask("🐇 Rabbit")
    }
    
    // Wait for all to complete
    Await.result(Future.sequence(Seq(lion, fox, rabbit)), 10.seconds)
    
    val duration = JDuration.between(start, Instant.now()).toMillis / 1000.0
    println(s"\nAll animals finished! Total time with $threadCount thread(s): $duration seconds")
  } finally {
    executorService.shutdown()
  }
}

// Run with different thread pool sizes
runWithThreads(1)  // Sequential execution
runWithThreads(3)  // Parallel execution


=== Running with 1 thread(s) ===

🦁 Lion starting work...
🦁 Lion finished!
🦊 Fox starting work...
🦊 Fox finished!
🐇 Rabbit starting work...
🐇 Rabbit finished!

All animals finished! Total time with 1 thread(s): 3.012 seconds

=== Running with 3 thread(s) ===

🦁 Lion starting work...
🦊 Fox starting work...
🐇 Rabbit starting work...
🐇 Rabbit finished!
🦁 Lion finished!
🦊 Fox finished!

All animals finished! Total time with 3 thread(s): 1.009 seconds


defined function cpuBoundTask
defined function runWithThreads

## 3. Deadlock Demonstration

This example shows a classic deadlock scenario where two threads each hold a resource the other needs.

**Note:** We won't actually run this in the notebook as it would deadlock. Instead, let's look at the code and understand it.

**File: `thread_deadlock.scala`**

In [5]:
// Print code only - don't run
println("""
import java.util.concurrent.locks.ReentrantLock

// Two locks representing resources
val meat = new ReentrantLock() // 🍖
val cheese = new ReentrantLock() // 🧀

// Lion thread - grabs meat first, then tries to get cheese
val lionThread = new Thread(() => {
  meat.lock()
  println("🦁 Lion grabs the 🍖 Meat!")
  Thread.sleep(100) 
  println("🦁 Lion wants the 🧀 Cheese...")
  cheese.lock() // This will block if Fox already has cheese
  println("🦁 Lion got the 🧀 Cheese too!")
  cheese.unlock()
  meat.unlock()
})

// Fox thread - grabs cheese first, then tries to get meat
val foxThread = new Thread(() => {
  cheese.lock()
  println("🦊 Fox grabs the 🧀 Cheese!")
  Thread.sleep(100)
  println("🦊 Fox wants the 🍖 Meat...")
  meat.lock() // This will block if Lion already has meat
  println("🦊 Fox got the 🍖 Meat too!")
  meat.unlock()
  cheese.unlock()
})

// Starting these creates deadlock:
// 1. Lion gets meat, Fox gets cheese
// 2. Lion waits for cheese (held by Fox)
// 3. Fox waits for meat (held by Lion)
// 4. Deadlock!
""")


import java.util.concurrent.locks.ReentrantLock

// Two locks representing resources
val meat = new ReentrantLock() // 🍖
val cheese = new ReentrantLock() // 🧀

// Lion thread - grabs meat first, then tries to get cheese
val lionThread = new Thread(() => {
  meat.lock()
  println("🦁 Lion grabs the 🍖 Meat!")
  Thread.sleep(100) 
  println("🦁 Lion wants the 🧀 Cheese...")
  cheese.lock() // This will block if Fox already has cheese
  println("🦁 Lion got the 🧀 Cheese too!")
  cheese.unlock()
  meat.unlock()
})

// Fox thread - grabs cheese first, then tries to get meat
val foxThread = new Thread(() => {
  cheese.lock()
  println("🦊 Fox grabs the 🧀 Cheese!")
  Thread.sleep(100)
  println("🦊 Fox wants the 🍖 Meat...")
  meat.lock() // This will block if Lion already has meat
  println("🦊 Fox got the 🍖 Meat too!")
  meat.unlock()
  cheese.unlock()
})

// Starting these creates deadlock:
// 1. Lion gets meat, Fox gets cheese
// 2. Lion waits for cheese (held by Fox)
// 3. Fox waits for meat (he

## 4. Exception Handling in Future Hierarchies

This example demonstrates how exceptions propagate through hierarchies of futures and explores the concept of orphaned tasks.

**File: `async_hierarchy_panics.scala`**

In [6]:
// Here we'll run a simplified version to demonstrate the key concepts
def runAsyncHierarchyDemo(): Unit = {
  println("🌍 World: Starting Hierarchy Demo")
  
  // Mammal branch with a bear that will fail
  val mammal = Future {
    println("  🐾 Mammal: Started")
    
    // The bear spawns fruit tasks then fails
    val bear = Future {
      println("    🐻 Bear: Started")
      
      // Bear's child tasks
      val apple = Future {
        println("      🍎 Apple: Started")
        Thread.sleep(500) // Long task
        println("      🍎 Apple: Finished")
      }
      
      val banana = Future {
        println("      🍌 Banana: Started")
        Thread.sleep(150)
        println("      🍌 Banana: Finished")
      }
      
      // Bear panics before apple can finish
      Thread.sleep(100)
      println("    🐻 Bear: Panicking now!")
      throw new RuntimeException("Bear panicked: bear hates apples!")
    }
    
    // Handle bear's exception
    try {
      Await.result(bear, 1.second)
      println("    🐻 Bear: Finished successfully")
    } catch {
      case e: Exception => 
        println(s"    🐻 Bear: Failed with: ${e.getMessage}")
    }
    
    println("  🐾 Mammal: Finished")
  }
  
  // Wait for the mammal branch to finish
  Await.result(mammal, 2.seconds)
  
  // Notice: We'll see the Banana finish, but the Apple might still be running
  // This demonstrates the "orphaned task" concept
  println("🌍 World: Hierarchy Demo Completed")
  println("Note: The Apple task may continue running after its parent (Bear) failed!")
}

runAsyncHierarchyDemo()

🌍 World: Starting Hierarchy Demo
  🐾 Mammal: Started
    🐻 Bear: Started
      🍎 Apple: Started
      🍌 Banana: Started
    🐻 Bear: Panicking now!
    🐻 Bear: Failed with: Bear panicked: bear hates apples!
  🐾 Mammal: Finished
🌍 World: Hierarchy Demo Completed
Note: The Apple task may continue running after its parent (Bear) failed!


defined function runAsyncHierarchyDemo

## 5. Best Practices: Future[Either] Pattern

This example demonstrates recommended patterns for working with Scala's Futures, focusing on proper error handling with the `Future[Either[Error, Success]]` pattern.

**File: `future.scala`**

In [7]:
// Domain classes for our example
case class Animal(name: String, species: String)
case class Diet(foodType: String, dailyAmount: Int)

// Error model with sealed trait
sealed trait ZooError {
  def message: String
}
case class DatabaseError(message: String) extends ZooError
case class ValidationError(message: String) extends ZooError

// Demo of the Future[Either] pattern
def demoFutureEither(): Unit = {
  println("\n=== Future[Either] Best Practice Demo ===\n")
  
  // Step 1: Service returning Future[Either]
  def findAnimal(name: String): Future[Either[ZooError, Animal]] = Future {
    println(s"🔍 Searching for animal: $name")
    
    name match {
      case "lion" => Right(Animal("Leo", "Lion"))
      case "tiger" => Right(Animal("Tigra", "Tiger"))
      case _ => Left(ValidationError(s"Unknown animal: $name"))
    }
  }
  
  // Step 2: Another service returning Future[Either]
  def getDiet(animal: Animal): Future[Either[ZooError, Diet]] = Future {
    println(s"🥩 Fetching diet for: ${animal.name} the ${animal.species}")
    
    animal.species match {
      case "Lion" => Right(Diet("Meat", 10))
      case "Tiger" => Right(Diet("Meat", 8))
      case _ => Left(ValidationError(s"No diet info for ${animal.species}"))
    }
  }
  
  // Step 3: Combining futures with for-comprehension
  def getAnimalWithDiet(name: String): Future[Either[ZooError, (Animal, Diet)]] = {
    for {
      // Find animal
      animalResult <- findAnimal(name)
      
      // If animal found, get diet (otherwise short-circuit)
      dietResult <- animalResult match {
        case Right(animal) => getDiet(animal).map(diet => diet.map(d => (animal, d)))
        case Left(error) => Future.successful(Left(error))
      }
    } yield dietResult
  }
  
  // Test with successful case
  val lionResult = getAnimalWithDiet("lion")
  Await.result(lionResult, 2.seconds) match {
    case Right((animal, diet)) => 
      println(s"✅ Success: ${animal.name} the ${animal.species} eats ${diet.foodType}")
    case Left(error) => 
      println(s"❌ Error: ${error.message}")
  }
  
  // Test with error case
  val unicornResult = getAnimalWithDiet("unicorn")
  Await.result(unicornResult, 2.seconds) match {
    case Right((animal, diet)) => 
      println(s"✅ Success: ${animal.name} the ${animal.species} eats ${diet.foodType}")
    case Left(error) => 
      println(s"❌ Error: ${error.message}")
  }
}

demoFutureEither()

      🍌 Banana: Finished
      🍎 Apple: Finished

=== Future[Either] Best Practice Demo ===

🔍 Searching for animal: lion
🥩 Fetching diet for: Leo the Lion
✅ Success: Leo the Lion eats Meat
🔍 Searching for animal: unicorn
❌ Error: Unknown animal: unicorn


defined class Animal
defined class Diet
defined trait ZooError
defined class DatabaseError
defined class ValidationError
defined function demoFutureEither

## 6. Running External Scala Files

We can also run our complete Scala examples using shell commands from the notebook:

In [8]:
// Run the await.scala example (if you want to run it externally)
// Uncomment to execute

"scala await.scala". !!

res8: String = """🦁 lionRuns() is called.
🦊 foxRuns() is called.
🦁 Lion is running!
🐇 rabbitRuns() is called.
🦊 Fox is running!
(All animals are running already in Scala Future!)
🐇 Rabbit is running!
All animals have finished running (even the rabbit 🐇, because Scala Future is eager)!

--- Lazy version ---
🦁 lazyLionRuns() is called.
🦊 lazyFoxRuns() is called.
🐇 lazyRabbitRuns() is called.
(No animals are running yet...)
🦊 Fox is running!
🦁 Lion is running!
All animals have finished running (except the rabbit 🐇, who never started)!
"""

## Key Differences: Scala vs Rust Async

| Aspect | Scala Future | Rust Async/Await |
|--------|-------------|------------------|
| Execution Model | **Eager**: Starts executing as soon as created | **Lazy**: Only starts when awaited |
| Type System | Uses `Future[T]` for all async results | Uses `impl Future<Output=T>` and other future types |
| Error Handling | Usually with `Try`, `Either`, or exceptions | With `Result<T, E>` return types |
| Cancellation | No built-in cancellation | Dropping a future can cancel it |
| Composition | `map`, `flatMap`, for-comprehensions | Async blocks, `.await` syntax |
| Runtime | Requires an `ExecutionContext` | Requires a runtime like Tokio |
| Resource Efficiency | Each future is a lightweight task | Zero-cost abstraction with state machines |

## Learning Resources

* [Introduction to scala-async](https://www.baeldung.com/scala/scala-async)

* [Working with Futures in Scala: A Quick Introduction](https://towardsdev.com/working-with-futures-in-scala-a-quick-introduction-9223703ab25e)

* [Synchronous Handling of Futures](https://www.baeldung.com/scala/synchronous-handling-of-futures)

### Sync, Async, Blocking and Non-Blocking | Rock the JVM

[![Sync, Async, Blocking and Non-Blocking | Rock the JVM](https://img.youtube.com/vi/Hlu-zYeNsSU/0.jpg)](https://www.youtube.com/watch?v=Hlu-zYeNsSU)

[Watch on YouTube](https://www.youtube.com/watch?v=Hlu-zYeNsSU)

---

### Adam Rosien - Async/Await for the Monadic Programmer | Scala Days 2023 Seattle

[![Adam Rosien - Async/Await for the Monadic Programmer | Scala Days 2023 Seattle](https://img.youtube.com/vi/OH5cxLNTTPo/0.jpg)](https://www.youtube.com/watch?v=OH5cxLNTTPo)

[Watch on YouTube](https://www.youtube.com/watch?v=OH5cxLNTTPo)

---

### Scala Programming - Introduction to Threads and Futures

[![Scala Programming - Introduction to Threads and Futures](https://img.youtube.com/vi/6b24sszy6Js/0.jpg)](https://youtu.be/6b24sszy6Js)

[Watch on YouTube](https://youtu.be/6b24sszy6Js)